In [24]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import time

In [2]:
#Load data and keep index id as "ind_id"
data = pd.read_csv("#Customer Invoices.csv", sep=";").reset_index().rename(columns={'index': 'ind_id'})

In [4]:
pd.set_option('display.max_columns', None)
#GLOBALS for making changing them easier
#Threshold for number of invoices a company must have to be included in the data
INVOICE_NR_THRESHOLD = 50
#Threshold for the number of days after due date when an invoice is considered late
LATENESS_THRESHOLD = 0
#Threshold for recency window. Used to limit the number prior invoices for calculations
# Using (len(data)+1) for all invoices, any random int for custom values
RECENCY_THRESHOLD = len(data) + 1

NameError: name 'data' is not defined

In [4]:
# Format dates to datetime objects
data['TransDate']= pd.to_datetime(data['TransDate'], format='%d.%m.%Y')
data['DueDate']= pd.to_datetime(data['DueDate'], format='%d.%m.%Y')
data['Closed']= pd.to_datetime(data['Closed'], format='%d.%m.%Y')
data['SettleTransDate']= pd.to_datetime(data['SettleTransDate'], format='%d.%m.%Y')

In [5]:
#Filter out not paid invoices
data = data[data.InvoiceState == 'F']
#Filtered out some questionable invoices
data = data[data.Invoice != 'EUR-i kursi muutus']
# concerting the amount from string to float
data['AmountEUR'] = data.AmountEUR.str.replace(',','.').astype('float')
# dropping columns that reflect duplicate data
data = data.drop(columns=['SettledAmountEUR', 'InvoiceState', 'DataAreaId'])

In [6]:
# dropping date equal to 1900.01.01 invoice AmountEUR smaller or equal than 0
data = data[data.AmountEUR >0]
data = data[data.SettleTransDate != '1900-01-01']
#Drop duplicates and unnecessary columns
data = data.drop(columns=['Closed', 'Type'])
data = data.drop_duplicates()
#Drop duplicate invoices if both invoice number and client ID are the same, keep those with different client ID
data = data[~data.duplicated(subset=['Invoice', 'Customer_NKey'])]

In [7]:
# customers list with invoices count greater or equal to INVOICE_NR_THRESHOLD
count = pd.DataFrame((data.groupby(data['Customer_NKey'])['Customer_NKey'].agg(['count']).sort_values(by=('count'), ascending=False)))
customer_list = count[count['count'] >= INVOICE_NR_THRESHOLD].index.values.tolist()
data_n1 = data[data.Customer_NKey.isin(customer_list)].copy(deep=True)

In [8]:
#Changing the order of columns
data_n1 = data_n1.iloc[:,[0, 1, 2, 3, 4, 6, 5]]

# Feature engineering

In [9]:
def check_current_status(dataset, selected_date):
    conds = [((dataset.DueDate < selected_date) & (dataset.SettleTransDate > selected_date)), ((dataset.DueDate < selected_date) & (dataset.SettleTransDate < selected_date))]  
    choices = [1, -1] # 1 is NOT paid, but IS LATE, -1 is Paid
    dataset['current_status'] = np.select(conds, choices, default=0) #0 = Not Paid But NOT late
    return dataset

In [10]:
def calculate_current_late_days(data_data, selected_date):
    dataset = data_data.copy(deep=True)
    dataset["temporary_late_days"] = selected_date - dataset.DueDate
    dataset["temporary_late_days"] = dataset["temporary_late_days"].apply(lambda x: int(x.days))
    dataset.loc[dataset['current_status'] == 0, 'days_late'] = 0
    dataset.loc[dataset['current_status'] == 1, 'days_late'] = dataset.temporary_late_days.apply(lambda x: x)
    dataset = dataset.drop(columns=['temporary_late_days'])
    return dataset

In [11]:
def filter_and_analyse(df_name, index_id_nr):
    
    ## Setting up and selecting dataframe
    global RECENCY_THRESHOLD
    #Select the current row
    selected_row_data = df_name.loc[index_id_nr]
    #Get Customer ID
    selected_Customer_NKey = selected_row_data["Customer_NKey"]
    #Get the date when invoice was created
    selected_TransDate = selected_row_data["TransDate"]
    #Get other invoices for the same client
    #Select only invoices prior to the invoice, invoices created on the same day not included
    selected_df = df_name[(df_name.TransDate < selected_TransDate) & (df_name.Customer_NKey == selected_Customer_NKey)].sort_values(by="TransDate", ascending=False)
    #Selected number of invoices for calculation
    selected_df = selected_df[selected_df.ind_id != index_id_nr].head(RECENCY_THRESHOLD)
    #Calculate rolling late days for invoices which are not yet closed
    selected_df = check_current_status(selected_df, selected_TransDate)
    selected_df = calculate_current_late_days(selected_df, selected_TransDate)
    
    
    ### For paid invoices
    selected_df_paid = selected_df[selected_df.SettleTransDate < selected_TransDate].copy(deep=True)
    ### For outstanding invoices
    selected_df_outstanding = selected_df[selected_df.SettleTransDate >= selected_TransDate].copy(deep=True)
    ## Seperation between early and late payments
    ### For paid invoices
    selected_df_paid["current_days_late"] = selected_df_paid["days_late"].apply(lambda x: abs(np.max([0, x])))
    selected_df_paid["current_days_early"] = selected_df_paid["days_late"].apply(lambda x: abs(np.min([0, x])))
    ### For outstanding invoices
    selected_df_outstanding["temp_days_late"] = selected_df_outstanding["DueDate"].apply(lambda x: selected_TransDate - x)
    selected_df_outstanding["temp_days_late"] = selected_df_outstanding["temp_days_late"].apply(lambda x: int(x.days))
    selected_df_outstanding["current_days_late"] = selected_df_outstanding["temp_days_late"].apply(lambda x: abs(np.max([0, x])))
    selected_df_outstanding = selected_df_outstanding.drop(columns=['temp_days_late'])

    #Unconditional late days:
    selected_df["current_days_late"] = selected_df["days_late"].apply(lambda x: abs(np.max([0, x])))
    selected_df["current_days_early"] = selected_df["days_late"].apply(lambda x: abs(np.min([0, x])))
    
    
     
    ## Getting the needed features
    #Totals and sums for all invoices
    #5
    f_total_number_invoices = len(selected_df)
    #6
    f_total_sum_invoices = selected_df.AmountEUR.sum()
    #7
    f_average_days_late_and_early = selected_df_paid.days_late.mean()
    #8
    f_average_days_late_total = selected_df_paid.current_days_late.mean()
    #9
    f_average_days_early_total = selected_df_paid.current_days_early.mean()
    #10
    try:
        f_last_invoice_paid_status = 1 if selected_df.iloc[0]["ind_id"] == selected_df_paid.iloc[0]["ind_id"] else 0
    except IndexError:
        f_last_invoice_paid_status = -1 #first-bill
    #11
    f_total_paid_invoices = len(selected_df_paid)
    #12
    f_sum_amount_paid_invoices = selected_df_paid["AmountEUR"].sum()
    #13
    f_total_invoices_late = len(selected_df_paid[(selected_df_paid.current_days_late > 0)])
    #14
    f_total_invoices_early = len(selected_df_paid[(selected_df_paid.current_days_early >= 0) & (selected_df_paid.current_days_late < 1)])
    #15
    f_sum_amount_late_invoices = selected_df_paid[selected_df_paid.current_days_late > 0]["AmountEUR"].sum()
    #16
    f_sum_amount_early_invoices = selected_df_paid[(selected_df_paid.current_days_early >= 0) & (selected_df_paid.current_days_late < 1)]["AmountEUR"].sum()
    #17
    f_total_outstanding_invoices = len(selected_df_outstanding)
    #18
    try:
        f_total_outstanding_late = sum(selected_df_outstanding.current_days_late > 0)
    except TypeError:
        f_total_outstanding_late = -9999
    #19
    f_sum_total_outstanding = selected_df_outstanding["AmountEUR"].sum()
    #20
    try:
        f_sum_late_outstanding = selected_df_outstanding[selected_df_outstanding.current_days_late > 0]["AmountEUR"].sum()
    except TypeError:
        f_sum_late_outstanding = -9999
    #21
    f_average_days_late = selected_df_paid[selected_df_paid.current_days_late > 0]["current_days_late"].mean()
    #22
    f_average_days_early = selected_df_paid[(selected_df_paid.current_days_early >= 0) & (selected_df_paid.current_days_late == 0)]["current_days_early"].mean()
    #23
    try:
        f_average_days_outstanding_late = selected_df_outstanding[selected_df_outstanding.current_days_late > 0]["current_days_late"].mean()
    except TypeError:
        f_average_days_outstanding_late = -9999
    #24
    f_std_days_late = selected_df_paid[selected_df_paid.current_days_late > 0]["current_days_late"].std()
    #25
    f_std_days_early = selected_df_paid[(selected_df_paid.current_days_early >= 0) & (selected_df_paid.current_days_late == 0)]["current_days_early"].std()
    #26
    try:
        f_std_days_outstanding_late = selected_df_outstanding[selected_df_outstanding.current_days_late > 0]["current_days_late"].std()
    except TypeError:
        f_std_days_outstanding_late = -9999
    #List all the results. Zeroes for indices 0-4 so feature # would match the positinal index in list.
    results = [0, 0, 0, 0, 0,
               f_total_number_invoices,
               f_total_sum_invoices,
                f_average_days_late_and_early,
                f_average_days_late_total,
                f_average_days_early_total,
                f_last_invoice_paid_status,
                f_total_paid_invoices,
                f_sum_amount_paid_invoices,
                f_total_invoices_late,
                f_total_invoices_early,
                f_sum_amount_late_invoices,
                f_sum_amount_early_invoices,
                f_total_outstanding_invoices,
                f_total_outstanding_late,
                f_sum_total_outstanding,
                f_sum_late_outstanding,
                f_average_days_late,
                f_average_days_early,
                f_average_days_outstanding_late,
                f_std_days_late,
                f_std_days_early,
                f_std_days_outstanding_late,
                ]
    
    return results

### Features 1-4

In [12]:
# #1 Unconditional days_late
data_n1["days_late"] = data_n1.SettleTransDate - data_n1.DueDate
data_n1["days_late"] = data_n1.days_late.apply(lambda x: int(x.days))

In [13]:
# #2 current_invoice_late_status aka is_late
data_n1["is_late"] = data_n1.days_late.apply(lambda x: 1 if x > LATENESS_THRESHOLD else 0)

In [14]:
# #3 days_between_created_and_paid
data_n1["days_between_created_and_paid"] = data_n1.SettleTransDate - data_n1.TransDate
data_n1["days_between_created_and_paid"] = data_n1.days_between_created_and_paid.apply(lambda x: int(x.days))

In [15]:
# #4 days_between_created_and_due
data_n1["days_between_created_and_due"] = data_n1.DueDate - data_n1.TransDate
data_n1["days_between_created_and_due"] = data_n1.days_between_created_and_due.apply(lambda x: int(x.days))

### Features 5-26

In [16]:
%%time
# #5-26: Creating temporary column with features 5-26 combined
# Took about 50 minutes to run it. 
data_n1["all_features"] = data_n1["ind_id"].apply(lambda x: filter_and_analyse(data_n1, x))

CPU times: total: 50min 20s
Wall time: 50min 28s


In [7]:
## Saved and loaded the data so last step can be skilled if necessary
#data_n1.to_csv("beforeseparation.csv")
data_n1= pd.read_csv("beforeseparation.csv", index_col=0)

In [8]:
#Since saving and loading to and from csv messed up the all_features list, here's fix:
def string_to_list(string):
    # input example (with brackets, commas and space): "[42, 42, 42]"
    string = string[1:len(string)-1]
    try:
        if len(string) != 0: 
            temp_list = string.split(", ")
            new_list = list(map(lambda x: (x), temp_list))
        else:
            new_list = []
    except:
        new_list = [-9998]
    return(new_list)

In [9]:
data_n1["all_features"] = data_n1.all_features.apply(lambda x: string_to_list(x))

In [10]:
list_of_column_names = [0,0,0,0,0,
                        "total_number_invoices", #5
                        "total_sum_invoices", #6
                        "average_days_late_and_early", #7
                        "average_days_late_total", #8
                        "average_days_early_total", #9
                        "last_invoice_paid_status", #10
                        "total_paid_invoices", #11,
                        "sum_amount_paid_invoices", #12
                        "total_invoices_late", #13
                        "total_invoices_early", #14
                        "sum_amount_late_invoices", #15
                        "sum_amount_early_invoices", #16
                        "total_outstanding_invoices", #17
                        "total_outstanding_late", #18
                        "sum_total_outstanding", #19
                        "sum_late_outstanding", #20
                        "average_days_late", #21
                        "average_days_early", #22
                        "average_days_outstanding_late", #23
                        "std_days_late", #24
                        "std_days_early", #25
                        "std_days_outstanding_late"] #26

In [11]:
for feature_index in range(5,26+1):
    data_n1[f"{list_of_column_names[feature_index]}"] = data_n1.all_features.apply(lambda x:float(x[feature_index]))

In [12]:
# # Check for nulls 
#data_n1.isnull().sum()

In [13]:
# Nulls are expected for last invoices and invoices which did not meet criteria:
# such as did not have any outstanding late invoices at the time of the creation of a invoice

In [14]:
data_n1 = data_n1.drop(columns=["all_features"])

In [15]:
# It should be safe to replace null values with zeros.
data_n1 = data_n1.fillna(0)

In [16]:
# Values -9999 were created for the function to work before. We can equal these to 0 as well. 
data_n1 = data_n1.replace(-9999, 0)
#data_n1[data_n1.isin([-9999]).any(1)]


### Features 27-32: ratios

In [17]:
# #27 ratio_1_late = Ratio of 13 over 11. (total_invoice_late / total_paid_invoices).
data_n1["ratio_1_late"] = data_n1.total_invoices_late / data_n1.total_paid_invoices

In [18]:
# #28 ratio_1_early = Ratio of 14 over 11. (total_invoices_early / total_paid_invoices)
data_n1["ratio_1_early"] = data_n1.total_invoices_early / data_n1.total_paid_invoices

In [19]:
# #29 ratio_2_late = Ratio of 15. over 12. (sum_amount_late_invoices / sum_amount_paid_invoices).
data_n1["ratio_2_late"] = data_n1.sum_amount_late_invoices / data_n1.sum_amount_paid_invoices

In [20]:
# #30 ratio_2_early = Ratio of 16. over 12. (sum_amount_early_invoices / sum_amount_paid_invoices).
data_n1["ratio_2_early"] = data_n1.sum_amount_early_invoices / data_n1.sum_amount_paid_invoices

In [21]:
# #31 ratio_3_late = Ratio of 18 over 17. (total_outstanding_late /total_outstanding_invoices).
data_n1["ratio_3_late"] = data_n1.total_outstanding_late / data_n1.total_outstanding_invoices

In [22]:
# #32 ratio_4_late = Ratio of 20. over 19. (sum_late_outstanding / sum_total_outstanding).
data_n1["ratio_4_late"] = data_n1.sum_late_outstanding / data_n1.sum_total_outstanding

In [19]:
# Dividing by zero creates new null values in same cases. Replacing with 0.
data_n1 = data_n1.fillna(0)

### Finishing up and exporting data

In [20]:
#data_n1.to_csv("invoice_data_with_features.csv")

In [44]:
data_n1

,ind_id,Customer_NKey,Invoice,TransDate,DueDate,SettleTransDate,AmountEUR,days_late,is_late,days_between_created_and_paid,days_between_created_and_due,total_number_invoices,total_sum_invoices,average_days_late_and_early,average_days_late_total,average_days_early_total,last_invoice_paid_status,total_paid_invoices,sum_amount_paid_invoices,total_invoices_late,total_invoices_early,sum_amount_late_invoices,sum_amount_early_invoices,total_outstanding_invoices,total_outstanding_late,sum_total_outstanding,sum_late_outstanding,average_days_late,average_days_early,average_days_outstanding_late,std_days_late,std_days_early,std_days_outstanding_late,ratio_1_late,ratio_1_early,ratio_2_late,ratio_2_early,ratio_3_late,ratio_4_late,days_late_unc
1369,1369,COCA_11388,538157,2020-03-27,2020-06-25,2020-09-30,34999.55,97,1,187,90,1981.0,28358743.24,53.787456,53.841812,0.054355,0.0,1435.0,21719614.51,1429.0,6.0,20683289.98,1036324.53,546.0,246.0,6639128.73,3044755.99,54.067880,13.000000,49.882114,16.108230,15.244671,21.944046,0.995819,0.004181,0.952286,0.047714,0.450549,0.458608,97
1370,1370,COCA_11388,538158,2020-03-27,2020-06-25,2020-09-30,17509.62,97,1,187,90,1981.0,28358743.24,53.787456,53.841812,0.054355,0.0,1435.0,21719614.51,1429.0,6.0,20683289.98,1036324.53,546.0,246.0,6639128.73,3044755.99,54.067880,13.000000,49.882114,16.108230,15.244671,21.944046,0.995819,0.004181,0.952286,0.047714,0.450549,0.458608,97
1371,1371,COCA_11388,538166,2020-03-30,2020-06-28,2020-09-30,4089.97,94,1,184,90,1984.0,28414361.79,53.787456,53.841812,0.054355,0.0,1435.0,21719614.51,1429.0,6.0,20683289.98,1036324.53,549.0,246.0,6694747.28,3044755.99,54.067880,13.000000,52.882114,16.108230,15.244671,21.944046,0.995819,0.004181,0.952286,0.047714,0.448087,0.454798,94
1372,1372,COCA_11388,538167,2020-03-30,2020-06-28,2020-09-30,8414.64,94,1,184,90,1984.0,28414361.79,53.787456,53.841812,0.054355,0.0,1435.0,21719614.51,1429.0,6.0,20683289.98,1036324.53,549.0,246.0,6694747.28,3044755.99,54.067880,13.000000,52.882114,16.108230,15.244671,21.944046,0.995819,0.004181,0.952286,0.047714,0.448087,0.454798,94
1373,1373,COCA_11388,538172,2020-03-30,2020-06-28,2020-09-30,1393.62,94,1,184,90,1984.0,28414361.79,53.787456,53.841812,0.054355,0.0,1435.0,21719614.51,1429.0,6.0,20683289.98,1036324.53,549.0,246.0,6694747.28,3044755.99,54.067880,13.000000,52.882114,16.108230,15.244671,21.944046,0.995819,0.004181,0.952286,0.047714,0.448087,0.454798,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54569,54569,COCA_10804,549376,2021-12-10,2022-01-24,2022-04-14,25688.52,80,1,125,45,3052.0,79822359.40,111.604035,111.996835,0.392801,0.0,2528.0,63884967.01,2272.0,256.0,56547733.05,7337233.96,524.0,387.0,15937392.39,12027403.06,124.616197,3.878906,74.165375,51.279290,5.332566,49.796618,0.898734,0.101266,0.885149,0.114851,0.738550,0.754666,80
54570,54570,COCA_10804,549923,2022-01-19,2022-03-05,2022-04-14,41844.24,40,1,85,45,3147.0,82867189.58,111.772497,112.179977,0.407480,0.0,2567.0,65278780.87,2300.0,267.0,57524623.15,7754157.72,580.0,456.0,17588408.71,13778371.55,125.202609,3.917603,89.497807,51.736776,5.214997,53.848138,0.895988,0.104012,0.881215,0.118785,0.786207,0.783378,40
54571,54571,COCA_10804,550243,2022-02-01,2022-03-18,2022-04-14,56008.47,27,1,72,45,3179.0,83888080.70,111.894921,112.302831,0.407910,0.0,2579.0,65673643.28,2309.0,270.0,57809890.52,7863752.76,600.0,512.0,18214437.42,15245059.29,125.434820,3.896296,88.685547,51.777702,5.189744,58.350053,0.895308,0.104692,0.880260,0.119740,0.853333,0.836977,27
54572,54572,COCA_10804,550265,2022-02-03,2022-03-20,2022-04-14,63190.91,25,1,70,45,3185.0,84236417.34,111.894921,112.302831,0.407910,0.0,2579.0,65673643.28,2309.0,270.0,57809890.52,7863752.76,606.0,512.0,18562774.06,15245059.29,125.434820,3.896296,90.685547,51.777702,5.189744,58.350053,0.895308,0.104692,0.880260,0.119740,0.844884,0.821271,25
